# Numerical representation. 
Much of the following is based on this: 

https://pythonnumericalmethods.studentorg.berkeley.edu/notebooks/chapter09.01-BaseN-and-Binary.html

In addition to the course book. 

Lets' load some libraries up front:

In [4]:
import numpy as np 
import scipy as sp  
import matplotlib.pyplot as plt
import sys

## From last time  


### The iterative map

$$
 x_{n+1}=\left\{\begin{array}{lcl}
 2 x_n, & & x_n\in [0,\frac{1}{2}] \\
 2x_n-1, &  & x_n\in (\frac{1}{2},1]\end{array} \right.
$$
By hand, calculate the first iterations, starting with $x_0=\frac{1}{10}$. Thereafter, calculate the 50 first iterations in the cell below. 


In [80]:

x = 1.0/10
for n in range(50):
    print(f'n={n}, x={x}')
    if x<= 0.5:
        x = 2*x
    else:
        x = 2*x - 1


n=0, x=0.1
n=1, x=0.2
n=2, x=0.4
n=3, x=0.8
n=4, x=0.6000000000000001
n=5, x=0.20000000000000018
n=6, x=0.40000000000000036
n=7, x=0.8000000000000007
n=8, x=0.6000000000000014
n=9, x=0.20000000000000284
n=10, x=0.4000000000000057
n=11, x=0.8000000000000114
n=12, x=0.6000000000000227
n=13, x=0.20000000000004547
n=14, x=0.40000000000009095
n=15, x=0.8000000000001819
n=16, x=0.6000000000003638
n=17, x=0.2000000000007276
n=18, x=0.4000000000014552
n=19, x=0.8000000000029104
n=20, x=0.6000000000058208
n=21, x=0.20000000001164153
n=22, x=0.40000000002328306
n=23, x=0.8000000000465661
n=24, x=0.6000000000931323
n=25, x=0.20000000018626451
n=26, x=0.40000000037252903
n=27, x=0.8000000007450581
n=28, x=0.6000000014901161
n=29, x=0.20000000298023224
n=30, x=0.4000000059604645
n=31, x=0.800000011920929
n=32, x=0.6000000238418579
n=33, x=0.20000004768371582
n=34, x=0.40000009536743164
n=35, x=0.8000001907348633
n=36, x=0.6000003814697266
n=37, x=0.20000076293945312
n=38, x=0.40000152587890625
n=39

## Number bases

Number representation in different bases, $\beta$:
$$
x = \sum_{k=-m}^{n} a_k \beta^k = a_n β^n + a_{n−1}β^{n−1} + \ldots + a_1β^1 + a_0β^0 + a_{−1}β^{−1} + a_{−2}β^{−2} + \ldots + a_{−m} β^{−m}
$$
where each of the coefficients $a_k$ are integers in the range $[0,\beta-1]$.

We will mostly think in decimal and binary bases. 

## From binary to decimal
Each binary number can be written 
$$
x(base\ 2) = \sum_{k=-m}^{n} a_k 2^k = a_n 2^n + a_{n−1}2^{n−1} + \ldots + a_12^1 + a_02^0 + a_{−1}2^{−1} + a_{−2}2^{−2} + \ldots + a_{−m} 2^{−m}
$$

Example:
$$
11001.1= 1\cdot2^4+1\cdot2^3+0\cdot2^2+0\cdot2^1+1\cdot2^0+1\cdot2^{-1}=16+8+1+0.5=25.5(base\ 10)
$$

In [178]:
def binary_to_decimal(binary_str): # Made by AI, be careful. 
    """
    Convert a binary string (including fractional part) to decimal.
    Example: '11001.1' -> 25.5
    """
    if '.' in binary_str:
        int_part, frac_part = binary_str.split('.')
    else:
        int_part, frac_part = binary_str, ''
    # Convert integer part without using int()
    decimal = 0
    #print(list(enumerate(reversed(int_part))))
    for idx, digit in enumerate(reversed(int_part)):
        if digit == '1':
            decimal += 2 ** idx
    #decimal = int(int_part, 2) if int_part else 0
    # Convert fractional part
    for idx, digit in enumerate(frac_part):
        if digit == '1':
            decimal += 2 ** -(idx + 1)
    return decimal

# Example usage:
binary='11001.1'
print(f"Binary {binary} in decimal {binary_to_decimal('11001.1')}")  # Output: 25.5
binary='101.101'
print(f"Binary {binary} in decimal {binary_to_decimal('101.101')}") # Output: 5.625

Binary 11001.1 in decimal 25.5
Binary 101.101 in decimal 5.625


## From decimal to binary
Repeating, since binary number can be written 
$$
x(base\ 2) =  a_n 2^n + a_{n−1}2^{n−1} + \ldots + a_12^1 + a_02^0 + a_{−1}2^{−1} + a_{−2}2^{−2} + \ldots + a_{−m} 2^{−m}=x(base\ 10)
$$

Example:
$$
11(base \ 10)= 8+2+1= 2^3+2^1+2^0=1011(base\ 2)
$$

In general, given a $x(base\ 10)$ number we need to find the coefficients in 
$$
x(base\ 10) = a_n 2^n + a_{n−1}2^{n−1} + \ldots + a_12^1 + a_02^0 + a_{−1}2^{−1} + a_{−2}2^{−2} + \ldots + a_{−m} 2^{−m}
$$

Split the $x(base\ 10)$ into the integer and decimal parts, $x(base\ 10)=I+D$, where $I\in \mathbb{Z}$ and $D\in [0,1)$

### The integer part
$$
\begin{array}{rcl}
I(base\ 10) &=& a_n 2^n + a_{n−1}2^{n−1} + \ldots + a_1 2^1 + a_0 2^0 \\
&=& 2\cdot\left[a_n 2^n-1 + a_{n−1}2^{n−2} + \ldots + a_1\right]+a_0\\
&=& 2\cdot q+r
\end{array}
$$
where $ q=mod(I(base\ 10),2)$ and $r$ is the remainder.

### The decimal part
$$
\begin{array}{rcl}
D(base\ 10) &=& a_{-1} 2^{-1} + a_{-2}2^{−2} + \ldots + a_{-3} 2^{-3} + \ldots \\
&\Downarrow& \\
2D (base \ 10) &=& a_{-1}2^0  + a_{-2}2^{−1} + \ldots + a_{-3} 2^{-2} + \ldots 
\end{array}
$$
The integer part of $2D$ is either 1 or 0, so $int(2D)=a_{-1}$.
Repeat.


### Algorithm with example

To convert a **base 10 (decimal)** number to **binary**, follow these steps:

#### 1. **Split the Number**
Separate the number into its **integer** and **fractional** parts.

**Example**:  
Decimal: `13.625`  
- Integer part: `13`  
- Fractional part: `0.625`

---

#### 2. **Convert the Integer Part**
Use repeated **division by 2**, and record the **remainders**:

```
13 ÷ 2 = 6 remainder 1  
6 ÷ 2 = 3 remainder 0  
3 ÷ 2 = 1 remainder 1  
1 ÷ 2 = 0 remainder 1
```

Write the remainders **bottom to top**:  
**Binary integer part = `1101`**

---

#### 3. **Convert the Fractional Part**
Use repeated **multiplication by 2**, and record the **integer part** of each result:

```
0.625 × 2 = 1.25 → 1  
0.25 × 2 = 0.5  → 0  
0.5 × 2 = 1.0   → 1
```

**Binary fractional part = `.101`**

---

#### ✅ Final Result
```plaintext
13.625 (decimal) = 1101.101 (binary)
```



In [179]:
def decimal_to_binary_fraction(x, precision=32, return_int=False):# Made by AI, be careful.
    """
    Convert a decimal number in [0, 1) to binary string.
    
    Parameters:
        x (float): Decimal number between 0 and 1.
        precision (int): Number of binary digits after the point.
    
    Returns:
        str: Binary representation as a string.
    """
    if not (0 <= x < 1):
        raise ValueError("Input must be in the range [0, 1).")
    
    binary = []
    for _ in range(precision):
        x *= 2
        bit = int(x)
        binary.append(str(bit))
        x -= bit
        if x == 0:
            break
    if return_int:
        out = '0.' + ''.join(binary)
    else:
        out = ''.join(binary)
    return out

decimal_to_binary_fraction(0.1)


'00011001100110011001100110011001'

## Representing numbers in the computer


In [ ]:
x = 15 # Example value
print(f"x = {x:.6f}") # 
mantissa, exponent = np.frexp(x) # np.frexp gives mantissa between [0.5, 1) will adjust in next lines
mantissa = mantissa * 2 # Adjust mantissa to be in [1, 2)
exponent = exponent - 1 # Adjust exponent accordingly


print(f"  mantissa: {mantissa:.6f}, exponent: {exponent}")
reconstructed = mantissa * 2.0**exponent
mantissa_bin = decimal_to_binary_fraction(mantissa-1)#np.binary_repr(mantissa - 1, width=54)
exponent_bin = np.binary_repr(exponent+1023)
print(f"  mantissa (binary, only significant): {mantissa_bin}")
mantissa_bin_padded = mantissa_bin.ljust(52, '0')
print(f"  mantissa (binary, padded to 5 digits): {mantissa_bin_padded}")
print(f"  exponent (binary): {exponent_bin}")
print(f"  reconstructed: {reconstructed:.6f}")


x = 15.000000
  mantissa: 1.875000, exponent: 3
  mantissa (binary, only significant): 111
  mantissa (binary, padded to 5 digits): 1110000000000000000000000000000000000000000000000000
  exponent (binary): 10000000010
  reconstructed: 15.000000


What will be the neighbouring numbers we can represent? The next 

In [181]:
# Convert mantissa_bin_padded to a list for mutability
mantissa_bin_list = list(mantissa_bin_padded)

# Add one to the last digit (binary addition with carry)
carry = 1
for i in range(len(mantissa_bin_list)-1, -1, -1):
    if mantissa_bin_list[i] == '0':
        mantissa_bin_list[i] = '1'
        carry = 0
        break
    else:
        mantissa_bin_list[i] = '0'
# If carry is still 1, it means overflow, but we ignore it for 53 bits

mantissa_bin_padded_plus1 = ''.join(mantissa_bin_list)
print("mantissa_bin_padded:      ", mantissa_bin_padded)
print("mantissa_bin_padded_plus1:", mantissa_bin_padded_plus1)

mantissa_bin_padded:       1110000000000000000000000000000000000000000000000000
mantissa_bin_padded_plus1: 1110000000000000000000000000000000000000000000000001


In [183]:
# Convert mantissa_bin_padded to a list for mutability
mantissa_bin_list = list(mantissa_bin_padded)

# Subtract one from the last digit (binary subtraction with borrow)
borrow = 1
for i in range(len(mantissa_bin_list)-1, -1, -1):
    if mantissa_bin_list[i] == '1':
        mantissa_bin_list[i] = '0'
        borrow = 0
        break
    else:
        mantissa_bin_list[i] = '1'
# If borrow is still 1, it means underflow, but we ignore it for 53 bits

mantissa_bin_padded_minus1 = ''.join(mantissa_bin_list)
print("mantissa_bin_padded:       ", mantissa_bin_padded)
print("mantissa_bin_padded_minus1:", mantissa_bin_padded_minus1)

mantissa_bin_padded:        1110000000000000000000000000000000000000000000000000
mantissa_bin_padded_minus1: 1101111111111111111111111111111111111111111111111111


In [189]:
# Reconstruct the numbers using the mantissa_bin_padded_minus1 and mantissa_bin_padded_plus1

def binary_fraction_to_decimal(bin_str):
    """Convert a binary string (fractional part only, e.g., '101') to decimal in [0, 1)."""
    return np.float64(sum(np.float64(int(bit)) * np.float64(2)**-(i+1) for i, bit in enumerate(bin_str)))

print(binary_fraction_to_decimal(mantissa_bin_padded_minus1))  # Should be close to 0.5
print(binary_fraction_to_decimal(mantissa_bin_padded_plus1))   # Should be close to 0.5

sign = np.float64(1)  # assuming positive numbers as in the example

# Reconstruct using mantissa_bin_padded_minus1
mantissa_minus1 =  1+binary_fraction_to_decimal(mantissa_bin_padded_minus1)
reconstructed_minus1 =sign * mantissa_minus1 * np.float64(2.0)**np.float64(exponent)

# Reconstruct using mantissa_bin_padded_plus1
mantissa_plus1 = 1 + binary_fraction_to_decimal(mantissa_bin_padded_plus1)
reconstructed_plus1 = sign * (mantissa_plus1) * np.float64(2.0)**np.float64(exponent)

print(f"minus: {mantissa_minus1:.17f}, pluss: {mantissa_plus1:.17f}")
print(f"Reconstructed (mantissa - 1): {reconstructed_minus1:.17f}")
print(f"Reconstructed (mantissa + 1): {reconstructed_plus1:.17f}")
print(f"Difference: {reconstructed_plus1 - reconstructed_minus1}")

# Check if they are exactly equal
if reconstructed_plus1 == reconstructed_minus1:
    print("The reconstructed values are exactly equal.")
else:
    print("The reconstructed values are different.")

0.8749999999999998
0.8750000000000002
minus: 1.87499999999999978, pluss: 1.87500000000000022
Reconstructed (mantissa - 1): 14.99999999999999822
Reconstructed (mantissa + 1): 15.00000000000000178
Difference: 3.552713678800501e-15
The reconstructed values are different.


## Machine accuracy

In [ ]:
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

`sys.float_info` provides detailed information about the properties of floating point numbers (type `float`) on your system, according to the IEEE 754 double-precision standard (which is what Python uses for `float`). Here’s what the main fields mean:

- **max**: Largest representable positive float (`1.7976931348623157e+308`)
- **min**: Smallest positive normalized float (`2.2250738585072014e-308`)
- **epsilon**: The difference between 1 and the next representable float (`2.220446049250313e-16`). This is the machine precision.
- **dig**: Number of decimal digits of precision (15)
- **mant_dig**: Number of bits in the mantissa (53)
- **max_exp**: Maximum exponent for base 2 (1024)
- **min_exp**: Minimum exponent for base 2 (-1021)
- **max_10_exp**: Maximum exponent for base 10 (308)
- **min_10_exp**: Minimum exponent for base 10 (-307)
- **radix**: Base of the exponent (2, since binary)
- **rounds**: Rounding mode (1 means round to nearest)

In summary, this tells you the range, precision, and behavior of floating point numbers in Python on your system.

The **mantissa** (also called the significand) and the **exponent base** are key components of floating-point numbers, which are used to represent real numbers in computers.

- **Mantissa (Significand):**  
    This is the part of a floating-point number that contains its significant digits. For example, in scientific notation, the number 6.022 × 10²³ has a mantissa of 6.022.

- **Exponent Base:**  
    This is the base used for the exponent part of the number. In most computer systems (following the IEEE 754 standard), the base is 2 (binary). So, a floating-point number is represented as:  
    **number = mantissa × (base)^(exponent)**  
    For example, 1.5 × 2³ = 12.

In summary, the mantissa determines the precision of the number, while the exponent (with its base) determines the scale or magnitude. In Python, the base is always 2 for floating-point numbers.

### The numpy library

In [51]:
import numpy as np
# NumPy provides info for float32, since sys.float_info is for Python's float (float64)
print(np.finfo(np.float32))
print(np.finfo(np.float64))

Machine parameters for float32
---------------------------------------------------------------
precision =   6   resolution = 1.0000000e-06
machep =    -23   eps =        1.1920929e-07
negep =     -24   epsneg =     5.9604645e-08
minexp =   -126   tiny =       1.1754944e-38
maxexp =    128   max =        3.4028235e+38
nexp =        8   min =        -max
smallest_normal = 1.1754944e-38   smallest_subnormal = 1.4012985e-45
---------------------------------------------------------------

Machine parameters for float64
---------------------------------------------------------------
precision =  15   resolution = 1.0000000000000001e-15
machep =    -52   eps =        2.2204460492503131e-16
negep =     -53   epsneg =     1.1102230246251565e-16
minexp =  -1022   tiny =       2.2250738585072014e-308
maxexp =   1024   max =        1.7976931348623157e+308
nexp =       11   min =        -max
smallest_normal = 2.2250738585072014e-308   smallest_subnormal = 4.9406564584124654e-324
------------------

In [159]:
np.spacing(1e9)

np.float64(1.1920928955078125e-07)

`np.spacing(1e9)` returns the smallest possible difference between two distinct floating point numbers around the value `1e9` (one billion), using NumPy's float64 precision.

The result, `1.1920928955078125e-07`, is the distance between `1e9` and the next larger representable float. This value is also known as the **machine epsilon** at that scale. It shows the precision limit of floating point arithmetic near `1e9`: any two numbers closer than this cannot be distinguished by a float64 variable.

Try with other values. 

In [5]:
np.spacing(1)  # This will give the spacing for float64, which is the default in Python

np.float64(2.220446049250313e-16)

## More on round off errors. 

Adding a small number to a large number and then subtract the larger, we should get back the original small one. 

In [9]:
# Add 10^x+small_number-10^x

small_number = 0.123456789
expon = 10 # Try different exponents. 

new_number = (10.**expon + small_number) - 10**expon

print(f"small number: {small_number}")
print(f"new number: {new_number}")
print(f"difference: {small_number - new_number}")
print(f"relative difference: {(small_number - new_number) / small_number}")


small number: 0.123456789
new number: 0.12345695495605469
difference: -1.6595605469016395e-07
relative difference: -1.3442440552229488e-06


### Example 9 on page 30
Presented in a slightly different way. 

Assuming we have the discrete map:
$$
x_{k+1}=(n+1)\cdot x_k -1, 
$$
assuming $n$ positive integer. This is a discrete map on the form 
$$
x_{k+1}=f(x_k). 
$$
The equilibrium points for this map, $x^*$, will be if we get back the same value as the one we put into $f(x)$. So we need to solve $x^*=f(x^*)$. In our case: 
$$
x^*=(n+1)\cdot x^* -1 \Rightarrow x^* =1/n
$$
So if we start with $x_0=1/n$ we should for each iteration get back $1/n$. 

In anticipation of what follows, let's go a bit further and check stability for this fixed point. Assuming we enter the fixes point and add a small perturbation
$$
x_{n+1}=f(x^*+\epsilon_k)\sim f(x^*)+\epsilon f'(x^*)+ O(\epsilon^2) = x^* + \epsilon_k f'(x^*)
$$
So the perturbation develops. 
$$ 
\begin{array}{clc}
k=0, \ & x_0=x^* + \epsilon, & f(x_0)\sim x^*+ \epsilon f'(x^*), \\
k=1, \ & x_1=x^* + \epsilon f'(x^*), & f(x_1)\sim x^*+ \epsilon f'(x^*)^2, \\
k=2, \ & x_2=x^* + \epsilon f'(x^*)^2, & f(x_2)\sim x^*+ \epsilon f'(x^*)^3, \\
& \vdots & \\
k=m, \ & x_2=x^* + \epsilon f'(x^*)^m, & f(x_m)\sim x^*+ \epsilon f'(x^*)^m+1, \\
\end{array}
$$
Hence the fixed point is stable if $|f'(x^*)|<1$.

In our example $f(x)=(n+1)x-1$ so $f'(x)=(n+1)$, and since $n$ is assumed positive, the fixed point is unstable. As will be be demonstrated. 

In [168]:
a = [ ] 
for n in range (1 ,17) : 
    x = 1/n 
    for k in range (10) : # do 10 iterations
        x = (n + 1) * x - 1 
    x_10= x
    for k in range (29) : # do 29 further iterations
        x = (n + 1) * x - 1
    x_29= x
    print("n =", n, " 1/n=", 1/n, " x_10 =", x_10, " x_29 =", x_29)

n = 1  1/n= 1.0  x_10 = 1.0  x_29 = 1.0
n = 2  1/n= 0.5  x_10 = 0.5  x_29 = 0.5
n = 3  1/n= 0.3333333333333333  x_10 = 0.3333333333139308  x_29 = -5592405.0
n = 4  1/n= 0.25  x_10 = 0.25  x_29 = 0.25
n = 5  1/n= 0.2  x_10 = 0.20000000179015842  x_29 = 65959556527001.77
n = 6  1/n= 0.16666666666666666  x_10 = 0.16666666069313285  x_29 = -1.9234215915856572e+16
n = 7  1/n= 0.14285714285714285  x_10 = 0.1428571343421936  x_29 = -1.3176245766935393e+18
n = 8  1/n= 0.125  x_10 = 0.125  x_29 = 0.125
n = 9  1/n= 0.1111111111111111  x_10 = 0.11111116045435665  x_29 = 4.934324553889585e+21
n = 10  1/n= 0.1  x_10 = 0.10000020942782539  x_29 = 3.322173065069967e+23
n = 11  1/n= 0.09090909090909091  x_10 = 0.09090867429040372  x_29 = -8.241284018680293e+24
n = 12  1/n= 0.08333333333333333  x_10 = 0.08333254844270876  x_29 = -1.581853859316291e+26
n = 13  1/n= 0.07692307692307693  x_10 = 0.07692660590305422  x_29 = 6.100455014607263e+27
n = 14  1/n= 0.07142857142857142  x_10 = 0.07142735197992223  

### Another example. 
Demonstrating that the order of doing calculations matters. 

Evaluate
$$
\frac{1.234}{0.1234}-\frac{1.234}{0.1233}
$$
In two different ways and get two different results. 
1. Directly, by performing the fractions and then subtract. 
2. Evaluate 
$$
1.234(\frac{1}{0.1234}-\frac{1}{0.1233})
$$

In [160]:
# In float 32 
print(np.float32(1.234/0.1234) - np.float32(1.234/0.1233))
print(1.234*(np.float32(1/0.1234) - np.float32(1/0.1233)))  # This demonstrates some of the pitfalls more dramatically

-0.008110046
-0.00811074


In [162]:
#In float 64
print(np.float64(1.234/0.1234) - np.float64(1.234/0.1233))
print(1.234*(np.float64(1/0.1234) - np.float64(1/0.1233)))  # This demonstrates some of the pitfalls more dramatically

-0.008110300081101585
-0.008110300081103212


In [ ]:
#What about float 16?
print(np.float16(1.234/0.1234) - np.float16(1.234/0.1233))
print(1.234*(np.float16(1/0.1234) - np.float16(1/0.1233)))  # This demonstrates some of the pitfalls more dramatically

-0.007812
-0.00964


### Timing
How we do calculations also changes how long time they take. Calculate 
$$
1^3+2^3+3^3\ldots n^3
$$
for a given $n$.


In [175]:
import time

# Method 1: Running total using a for loop
Total = 0
n=101
print(f"n={n}")
start1 = time.time()
for i in range(1, n):
    Total += i**3
end1 = time.time()
print("Sum using running total:", Total)
print("Time for running total:", end1 - start1, "seconds")

    # Method 2: Build a list of perfect cubes and sum with sum()
cubes = []
start2 = time.time()
for i in range(1, n):
    cubes.append(i**3)
sum_cubes = sum(cubes)
end2 = time.time()

print("Sum using sum() on list:", sum_cubes)
print("Time for sum() on list:", end2 - start2, "seconds")
print("time difference:", (end2 - start2)-(end1 - start1), "seconds")
print("time difference in percent:", ((end2 - start2)-(end1 - start1)) / (end1 - start1) * 100, "%")


n=101
Sum using running total: 25502500
Time for running total: 0.0001308917999267578 seconds
Sum using sum() on list: 25502500
Time for sum() on list: 0.00013494491577148438 seconds
time difference: 4.0531158447265625e-06 seconds
time difference in percent: 3.096539162112933 %


## Computational Complexity of Operations

Computational complexity describes how the time (or number of steps) required to perform an operation grows as the size of the input increases. Here are some common complexities:

- **O(1): Constant time** — The operation takes the same amount of time regardless of input size. Example: accessing an element in a list by index.
- **O(n): Linear time** — The time grows linearly with input size. Example: summing all elements in a list.
- **O(n²): Quadratic time** — The time grows with the square of the input size. Example: nested loops over a list.

Let's demonstrate this with a numerical example by timing three operations: accessing an element, summing a list, and a nested loop.


In [73]:

import time
import numpy as np
import bisect
import time

n = 10**6
lst = list(range(n))

# O(log n): Binary search for an element
target = n - 1
start = time.time()
index = bisect.bisect_left(lst, target)
end = time.time()
print("O(log n) binary search time:", end - start, "seconds")

n = 10**6
lst = list(range(n))

# O(1): Accessing an element
start = time.time()
x = lst[n//2]
end = time.time()
print("O(1) access time:", end - start, "seconds")

# O(n): Summing all elements
start = time.time()
total = sum(lst)
end = time.time()
print("O(n) sum time:", end - start, "seconds")

# O(log n): Binary search for an element
target = n - 1
start = time.time()
index = bisect.bisect_left(lst, target)
end = time.time()
print("O(log n) binary search time:", end - start, "seconds")

# O(n^2): Nested loop (warning: can be slow for large n)
n_small = 1000  # Use a smaller n for quadratic example
lst_small = list(range(n_small))
start = time.time()
count = 0
for i in lst_small:
    for j in lst_small:
        count += i * j
end = time.time()
print("O(n^2) nested loop time:", end - start, "seconds")

O(log n) binary search time: 4.00543212890625e-05 seconds
O(1) access time: 3.910064697265625e-05 seconds
O(n) sum time: 0.006695985794067383 seconds
O(log n) binary search time: 3.4809112548828125e-05 seconds
O(n^2) nested loop time: 0.07391166687011719 seconds


**Task** Try this with increasing dimension and see how these numbers change.

## Theory: Why These Examples Have Their Complexity

- **O(1): Constant Time**  
  Accessing an element in a list by its index (e.g., `lst[n//2]`) is a constant-time operation. No matter how large the list is, Python can directly retrieve the value at any index without scanning the list. This is because lists are implemented as arrays in memory.

- **O(n): Linear Time**  
  Summing all elements in a list (e.g., `sum(lst)`) requires visiting each element exactly once. As the list grows, the time taken grows proportionally. If you double the size of the list, the time to sum it roughly doubles.

- **O(log n): Logarithmic Time**  
  Operations like binary search (`bisect.bisect_left(lst, target)`) have logarithmic complexity. Each step cuts the search space in half, so the number of steps grows with the logarithm of the input size. For example, searching in a list of 1,000,000 elements takes only about 20 steps, since $2^{20} \approx 1,000,000$. This makes logarithmic algorithms very efficient for large datasets.

- **O(n²): Quadratic Time**  
  A nested loop over a list (e.g., `for i in lst: for j in lst: ...`) means that for every element in the outer loop, you iterate over every element in the inner loop. If the list has `n` elements, you perform `n * n = n²` operations. This means the time increases very rapidly as the list size increases.

These examples illustrate how the structure of your code determines how well it will scale to large inputs.

## Keep in mind the truncation error
In NumPy, functions like sin, cos, and exp are evaluated using vectorized operations that are highly optimized and implemented in compiled C code under the hood for performance, more advanced methods than Taylor expansions. Still there will be truncation errors in these evaluations. Example in the next cell.  

In [ ]:
import numpy as np
import math

def taylor_sin(x, terms=10): # Compute sin(x) using Taylor series to a specified number of terms
    result = 0
    for n in range(terms):
        term = ((-1)**n) * (x**(2*n + 1)) / math.factorial(2*n + 1)
        result += term
    return result

# Compare with NumPy's sin
x = np.pi/4 # 45 degrees in radians
print(f"Taylor series: {taylor_sin(x, terms=2)}") # Taylor series approximation
print(f"NumPy sin:     {np.sin(x)}") # NumPy's implementation
print(f"Exact value:   {np.sqrt(2)/2}") # Exact value for sin(π/4)

Taylor series: 0.7046526512091675
NumPy sin:     0.7071067811865475
Exact value:   0.7071067811865476


Notice how the NumPy sin and "exact" value differs on last digit. 